In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

#550eV - 11.5eV CH9
#update nov Kilian binning (36,18)
fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CR_ALLCH_diatomic.root");en="550eV";
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_550eV_CL_ALLCH_diatomic.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CR_ALLCH_diatomic.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S-C3H3F3O_550eV_CL_ALLCH_diatomic.root")
CH="ALL_CH";
# CH="CH14";

# # 546eV - 8.2eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CR_ALLCH_diatomic.root");en="546eV";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_546eV_CL_ALLCH_diatomic.root")
# CH="ALL_CH";
# # CH="CH14";

# # 544eV - 6.1eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CR_ALLCH_diatomic.root");en="544eV";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_544.5eV_CL_ALLCH_diatomic.root")
# CH="ALL_CH";
# # CH="CH14";

# # 542eV - 4eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CR_ALLCH_diatomic.root");en="542eV";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_542.5eV_CL_ALLCH_diatomic.root")
# CH="ALL_CH";
# # CH="CH14";

# # 541eV 3eV CH9
# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CR_ALLCH_diatomic.root");en="541eV";
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R-C3H3F3O_541.5eV_CL_ALLCH_diatomic.root")
# CH="ALL_CH";
# # CH="CH14";


if CH=="ALL_CH":
    # path = "angular_distr_el/"+CH+"/test"
    path = "angular_distr_el/"+CH+"/"
    loc=path

else:
    path = "angular_distr_el/"+CH+"/"
    IDs = ["ID_ALL_mol_e0_valid/EN_gate/","ID1_mol_e0_valid/EN_gate/","ID2_mol_e0_valid/EN_gate/","ID3_mol_e0_valid/EN_gate/"]
    loc=path+IDs[0]

#new uproot4
# https://uproot.readthedocs.io/en/latest/basic.html
# d=dict(fileRCR[loc].classnames()) # shows the type of each item
# k=fileRCR[loc].keys() #same as .classnames()
# check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# d1=type(check)
# check.all_members; #all members of the single histogram

histRCR = fileRCR[loc]
histRCL = fileRCL[loc]
if en == "550eV":
    histSCR = fileSCR[loc]
    histSCL = fileSCL[loc]
else:
    fileSCR = None
    fileSCL = None

In [ ]:
#theory curves:
# R 500-4.50 random : *cos(x/180*3.1415926)/(1-1.124/2*(3*cos(x/180*3.1415926)*cos(x/180*3.1415926)-1)/2)

In [ ]:
if CH=="ALL_CH":
    print("ok ALL")
    #NOTE ALL doesn´t have a polarization plane cos(theta) becuase CH9 And CH11 are three bodies breakups
    a=16 #select resolution of the imported graph for the ALL comnbined.

    MFPAD_RCR_all,MFPAD_RCR_en_all,ctheta_RCR_all,ctheta_RCR_err_all,PECD_axis,ctheta_axis,ctheta_axis_red=fugi.import_PECD3D(fileRCR, loc, a=a,  full=True)
    MFPAD_RCL_all,MFPAD_RCL_en_all,ctheta_RCL_all,ctheta_RCL_err_all=fugi.import_PECD3D(fileRCL, loc, a=a)   

    #normalise before ANY PECD
    MFPAD_RCR_all_norm=fugi.normalise_with_err(MFPAD_RCR_all)
    MFPAD_RCL_all_norm=fugi.normalise_with_err(MFPAD_RCL_all)
    ctheta_RCR_all_norm, ctheta_RCR_err_all_norm=fugi.normalise_with_err(ctheta_RCR_all,ctheta_RCR_err_all)
    ctheta_RCL_all_norm, ctheta_RCL_err_all_norm=fugi.normalise_with_err(ctheta_RCL_all,ctheta_RCL_err_all)
    
    #Combining the two polarizations: single intesity fit
    ctheta_R_all=np.add(ctheta_RCR_all,np.flip(ctheta_RCL_all))
    ctheta_R_all_norm=np.add(ctheta_RCR_all_norm,np.flip(ctheta_RCL_all_norm))
    ctheta_R_all_err=fugi.error_calc(ctheta_RCR_err_all,np.flip(ctheta_RCL_err_all)) #SUM PROPAGATION
    
    #PECD between two diffrent helicities
    cPECD_R_all =  np.divide(np.subtract(ctheta_RCR_all_norm,ctheta_RCL_all_norm),np.add(ctheta_RCR_all_norm,ctheta_RCL_all_norm))
    errPECD_R_all = fugi.error_calc(ctheta_RCR_all,ctheta_RCL_all)

    #ALL channels combined: polarization doesn´t make sense because of CH9 and CH11 three bodies
    mPECD_R_all = np.divide(np.subtract(MFPAD_RCR_all_norm,MFPAD_RCL_all_norm),np.add(MFPAD_RCR_all_norm,MFPAD_RCL_all_norm))

    if fileSCL and fileSCR is not None:
        MFPAD_SCR_all,MFPAD_SCR_en_all,ctheta_SCR_all,ctheta_SCR_err_all=fugi.import_PECD3D(fileSCR, loc, a=a)
        MFPAD_SCL_all,MFPAD_SCL_en_all,ctheta_SCL_all,ctheta_SCL_err_all=fugi.import_PECD3D(fileSCL, loc, a=a)

        #normalise before ANY PECD
        MFPAD_SCR_all_norm=fugi.normalise_with_err(MFPAD_SCR_all)
        MFPAD_SCL_all_norm=fugi.normalise_with_err(MFPAD_SCL_all)
        ctheta_SCR_all_norm, ctheta_SCR_err_all_norm=fugi.normalise_with_err(ctheta_SCR_all,ctheta_SCR_err_all)
        ctheta_SCL_all_norm, ctheta_SCL_err_all_norm=fugi.normalise_with_err(ctheta_SCL_all,ctheta_SCL_err_all)

        ctheta_S_all=np.add(ctheta_SCR_all,np.flip(ctheta_SCL_all))
        ctheta_S_all_norm=np.add(ctheta_SCR_all_norm,np.flip(ctheta_SCL_all_norm))
        ctheta_S_all_err=fugi.error_calc(ctheta_SCR_err_all,np.flip(ctheta_SCL_err_all)) #SUM PROPAGATION

        #single PECD on just S enantiomer
        cPECD_S_all =  np.divide(np.subtract(ctheta_SCR_all_norm,ctheta_SCL_all_norm),np.add(ctheta_SCR_all_norm,ctheta_SCL_all_norm))
        errPECD_S_all = fugi.error_calc(ctheta_SCR_all,ctheta_SCL_all)
    
        #for 550eV total cos and PECD
        ctheta_all=np.add(ctheta_R_all,np.flip(ctheta_S_all))
        ctheta_all_err=fugi.error_calc(ctheta_R_all_err,np.flip(ctheta_S_all_err)) #SUM PROPAGATION

        cPECD_all =  np.divide(np.subtract(ctheta_R_all_norm,ctheta_S_all_norm),np.add(ctheta_R_all_norm,ctheta_S_all_norm))
        errPECD_all = fugi.error_calc(ctheta_R_all,ctheta_S_all,ctheta_R_all_err)

        mPECD_S_all = np.divide(np.subtract(MFPAD_SCR_all_norm,MFPAD_SCL_all_norm),np.add(MFPAD_SCR_all_norm,MFPAD_SCL_all_norm))

In [ ]:
if CH=="CH14":
    print("ok CH14")
    MFPAD_RCR,MFPAD_RCR_en,ctheta_RCR,ctheta_RCR_err,ctheta_pol_RCR,ctheta_pol_RCR_err,PECD_axis,ctheta_axis,ctheta_axis_red=fugi.import_PECD3D_cos(fileRCR,loc,full=True)
    MFPAD_RCL,MFPAD_RCL_en,ctheta_RCL,ctheta_RCL_err,ctheta_pol_RCL,ctheta_pol_RCL_err=fugi.import_PECD3D_cos(fileRCL, loc)

    if fileSCL and fileSCR is not None:    
        MFPAD_SCR,MFPAD_SCR_en,ctheta_SCR,ctheta_SCR_err,ctheta_pol_SCR,ctheta_pol_SCR_err=fugi.import_PECD3D_cos(fileSCR, loc)
        MFPAD_SCL,MFPAD_SCL_en,ctheta_SCL,ctheta_SCL_err,ctheta_pol_SCL,ctheta_pol_SCL_err=fugi.import_PECD3D_cos(fileSCL, loc)

    #normalise before ANY PECD
    ctheta_RCR_norm, ctheta_RCR_err_norm=fugi.normalise_with_err(ctheta_RCR,ctheta_RCR_err)
    ctheta_RCL_norm, ctheta_RCL_err_norm=fugi.normalise_with_err(ctheta_RCL,ctheta_RCL_err)
    ctheta_pol_RCR_norm, ctheta_pol_RCR_err_norm=fugi.normalise_with_err(ctheta_pol_RCR,ctheta_pol_RCR_err)
    ctheta_pol_RCL_norm, ctheta_pol_RCL_err_norm=fugi.normalise_with_err(ctheta_pol_RCL,ctheta_pol_RCL_err)
    
    #Combining the two polarizations: single intesity fit
    ctheta_R=np.add(ctheta_RCR,np.flip(ctheta_RCL))
    ctheta_R_norm=np.add(ctheta_RCR_norm,np.flip(ctheta_RCL_norm))
    ctheta_R_err=fugi.error_calc(ctheta_RCR_err,np.flip(ctheta_RCL_err))

    #Combining the two polarizations: single intesity fit POL PLANE
    ctheta_R_pol=np.add(ctheta_pol_RCR,np.flip(ctheta_pol_RCL))
    ctheta_R_pol_norm=np.add(ctheta_pol_RCR_norm,np.flip(ctheta_pol_RCL_norm))
    ctheta_R_pol_err= fugi.error_calc(ctheta_pol_RCR_err,np.flip(ctheta_pol_RCL_err))

    #PECD between two diffrent helicities
    cPECD_R =  np.divide(np.subtract(ctheta_RCR_norm,ctheta_RCL_norm),np.add(ctheta_RCR_norm,ctheta_RCL_norm))
    errPECD_R = fugi.error_calc(ctheta_RCR,ctheta_RCL)

    #PECD between two diffrent helicities POL PLANE
    cPECD_R_pol =  np.divide(np.subtract(ctheta_pol_RCR_norm,ctheta_pol_RCL_norm),np.add(ctheta_pol_RCR_norm,ctheta_pol_RCL_norm))
    errPECD_R_pol = fugi.error_calc(ctheta_pol_RCR,ctheta_pol_RCL)

    #ALL channels combined: polarization doesn´t make sense because of CH9 and CH11 three bodies
    mPECD_R = np.divide(np.subtract(MFPAD_RCR,MFPAD_RCL),np.add(MFPAD_RCR,MFPAD_RCL))

    if fileSCL and fileSCR is not None:
        ctheta_SCR_norm, ctheta_SCR_err_norm=fugi.normalise_with_err(ctheta_SCR,ctheta_SCR_err)
        ctheta_SCL_norm, ctheta_SCL_err_norm=fugi.normalise_with_err(ctheta_SCL,ctheta_SCL_err)
        ctheta_pol_SCR_norm, ctheta_pol_SCR_err_norm=fugi.normalise_with_err(ctheta_pol_SCR,ctheta_pol_SCR_err)
        ctheta_pol_SCL_norm, ctheta_pol_SCL_err_norm=fugi.normalise_with_err(ctheta_pol_SCL,ctheta_pol_SCL_err)

        ctheta_S=np.add(ctheta_SCR_norm,np.flip(ctheta_SCL_norm))
        ctheta_S_norm=np.add(ctheta_SCR_norm,np.flip(ctheta_SCL_norm))
        ctheta_S_err=fugi.error_calc(ctheta_SCR_err,np.flip(ctheta_SCL_err))

        #for 550eV total cos and PECD two enantiomers POL PLANE
        ctheta_S_pol=np.add(ctheta_pol_SCR,np.flip(ctheta_pol_SCL))
        ctheta_S_pol_norm=np.add(ctheta_pol_SCR_norm,np.flip(ctheta_pol_SCL_norm))
        ctheta_S_pol_err=fugi.error_calc(ctheta_pol_SCR_err,np.flip(ctheta_pol_SCL_err)) #SUM PROPAGATION

        cPECD_S=np.divide(np.subtract(ctheta_SCR_norm,ctheta_SCL_norm),np.add(ctheta_SCR_norm,ctheta_SCL_norm))
        errPECD_S=fugi.error_calc(ctheta_SCR,ctheta_SCL)

        cPECD_S_pol=np.divide(np.subtract(ctheta_pol_SCR_norm,ctheta_pol_SCL_norm),np.add(ctheta_pol_SCR_norm,ctheta_pol_SCL_norm))
        errPECD_S_pol=fugi.error_calc(ctheta_pol_SCR,ctheta_pol_SCL)

        #for 550eV total cos and PECD two enantiomers
        ctheta=np.add(ctheta_R_norm,np.flip(ctheta_S_norm))
        ctheta_err=fugi.error_calc(ctheta_R_err,np.flip(ctheta_S_err)) #SUM PROPAGATION

        #for 550eV total cos and PECD two enantiomers
        ctheta_pol=np.add(ctheta_R_pol,np.flip(ctheta_S_pol))
        ctheta_pol_err=fugi.error_calc(ctheta_R_pol_err,np.flip(ctheta_S_pol_err)) #SUM PROPAGATION


        cPECD =  np.divide(np.subtract(ctheta_R_norm,ctheta_S_norm),np.add(ctheta_R_norm,ctheta_S_norm))
        errPECD = fugi.error_calc(ctheta_R,ctheta_S,ctheta_R_err,ctheta_S_err, error_type="PECD")
        
        cPECD_pol =  np.divide(np.subtract(ctheta_R_pol_norm,ctheta_S_pol_norm),np.add(ctheta_R_pol_norm,ctheta_S_pol_norm))
        errPECD_pol = fugi.error_calc(ctheta_R_pol,ctheta_S_pol,ctheta_R_pol,ctheta_S_pol,error_type="PECD")
        
        mPECD_S = np.divide(np.subtract(MFPAD_SCR,MFPAD_SCL),np.add(MFPAD_SCR,MFPAD_SCL))

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

if CH == "ALL_CH":
    x_data = ctheta_axis_red;y_data = ctheta_RCL_all;y_err = ctheta_RCL_err_all
elif fileSCL and fileSCR is not None and CH == "ALL_CH":
    x_data = ctheta_axis_red;y_data = ctheta_SCL_all;y_err = ctheta_SCL_err_all   
elif fileSCL and fileSCR is not None and CH == "CH14":
    x_data = ctheta_axis_red;y_data = ctheta_SCL;y_err = ctheta_SCL_err   
else:
    x_data = ctheta_axis_red;y_data = ctheta_RCL;y_err = ctheta_RCL_err

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
if fileSCL and fileSCR is not None:
    ax1.set_title("S CL "+CH+" "+en)
else:
    ax1.set_title("R CL "+CH+" "+en)
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)


print("CR fitting and covarience \n", m1, "\n")

if CH == "ALL_CH":
    x_data = ctheta_axis_red;y_data = ctheta_RCR_all;y_err = ctheta_RCR_err_all
elif fileSCL and fileSCR is not None and CH == "ALL_CH":
    x_data = ctheta_axis_red;y_data = ctheta_SCR_all;y_err = ctheta_SCR_err_all
elif fileSCL and fileSCR is not None and CH == "CH14":
    x_data = ctheta_axis_red;y_data = ctheta_SCR;y_err = ctheta_SCR_err   
else:
    x_data = ctheta_axis_red;y_data = ctheta_RCR;y_err = ctheta_RCR_err

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
if fileSCL and fileSCR is not None:
    ax2.set_title("S CR "+CH+" "+en)
else:
    ax2.set_title("R CR "+CH+" "+en)
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CR fitting  and covarience \n", m2)

fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_single_hel_"+CH+"_"+en+"_exp.png", figsize=(20, 10), dpi=144, transparent=False)

In [ ]:
#INTESNITY OF THE COS(THETA)
#COMBINED ELLICITIES
import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

if fileSCL and fileSCR is not None:
    if CH == "ALL_CH":
        x_data = ctheta_axis_red;y_data = ctheta_all;y_err = ctheta_all_err
    else:
        x_data = ctheta_axis_red;y_data = ctheta;y_err = ctheta_err
else:
    if CH == "ALL_CH":
        x_data = ctheta_axis_red;y_data = ctheta_R_all;y_err = ctheta_R_all_err
    else:
        x_data = ctheta_axis_red;y_data = ctheta_R;y_err = ctheta_R_err

m=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m,ax,custom_xlim)
if fileSCL and fileSCR is not None:
    if CH == "ALL_CH":
        ax.set_title("R + S "+CH+" "+en)
    else:
        ax.set_title("S CL+CR "+CH+" "+en)
else:
    ax.set_title("R CL+CR "+CH+" "+en)

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax.set_aspect(aspect)

m
fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/cos(th)x_combi_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

if fileSCL and fileSCR is not None:
    if CH == "ALL_CH":
        x_data = ctheta_axis_red;y_data = cPECD_all;y_err = errPECD_all
    else:
        x_data = ctheta_axis_red;y_data = cPECD;y_err = errPECD
else:
    if CH == "ALL_CH":
        x_data = ctheta_axis_red;y_data = cPECD_R_all;y_err = errPECD_R_all
    else:
        x_data = ctheta_axis_red;y_data = cPECD_R;y_err = errPECD_R

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

if fileSCL and fileSCR is not None:
    if CH == "ALL_CH":
        ax.set_title("R + S all dir "+CH+" "+en)
    else:
        ax.set_title("S CL+CR all dir "+CH+" "+en)
else:
    ax.set_title("R CL+CR all dir "+CH+" "+en)

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_random_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
if CH == "CH14":
    #PECD INTERPOLATION POL
    # import functions_minuit as fumi

    fig,ax= plt.subplots()

    custom_xlim = (-1, 1)
    custom_ylim = (-0.15, 0.15)

    # limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
    limits=[(-1, 1), (-1, 2)] #PECD2

    if fileSCL and fileSCR is not None:
        x_data = ctheta_axis_red;y_data = cPECD_pol;y_err = errPECD_pol    
    else:
        x_data = ctheta_axis_red;y_data = cPECD_R_pol;y_err = errPECD_R_pol

    m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
    fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

    if fileSCL and fileSCR is not None:
    if CH == "ALL_CH":
        ax.set_title("R + S pol "+CH+" "+en)
    else:
        ax.set_title("S CL+CR pol "+CH+" "+en)
    else:
        ax.set_title("R CL+CR pol "+CH+" "+en)

    ax.set_xlabel('cos(\u03D1) [adm]')
    ax.set_ylabel('PECD')

    fit_info = []
    for p, v, e in zip(m.parameters, m.values, m.errors):
        fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

    ax.legend(title="\n".join(fit_info), loc="best")

    aspect=(custom_xlim[1]/custom_ylim[1])
    ax.set_aspect(aspect)

    print(m)

    fig.savefig("../PYTHON_graphs/OUTPUTS/Diatomic/PECD_pol_"+CH+"_"+en+"_exp.png", figsize=(12, 10), dpi=144, transparent=False)

In [ ]:
"""
OUTPUT for old OriginPro graphs
expressed in THETA -> np.flipped!!!
 
'r' open for reading (default)
'w' open for writing, truncating the file first     # file1.truncate(0)
'x' open for exclusive creation, failing if the file already exists
'a' open for writing, appending to the end of the file if it exists
"""
file1 = open("OUTPUTS/EXPORT_origin/export_origin_"+CH+"_"+en+".txt","w")
file1.write("PECD outptus in 15 bins avaraged from 30 bin \n") 
out=[]

out.append(np.arccos(np.linspace(-0.9333,0.9333,15))*180/np.pi)

if CH == "CH14":
    out.append(fugi.bin_ndarray(np.flip(cPECD_R),new_shape=(15,), operation='mean').tolist())
    out.append(fugi.bin_ndarray(np.flip(errPECD_R),new_shape=(15,), operation='mean').tolist())
    out.append(fugi.bin_ndarray(np.flip(cPECD_R_pol),new_shape=(15,), operation='mean').tolist())
    out.append(fugi.bin_ndarray(np.flip(errPECD_R_pol),new_shape=(15,), operation='mean').tolist())
    file1.write("theta \t cPECD_R \t err \t cPECD_R_pol \t err \t") 

    if fileSCL and fileSCR is not None:
        out.append(fugi.bin_ndarray(np.flip(cPECD_S),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(errPECD_S),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(cPECD_S_pol),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(errPECD_S_pol),new_shape=(15,), operation='mean').tolist())
        file1.write("\t cPECD_S \t err \t cPECD_S_pol \t err \t") 

elif CH == "ALL_CH":
    out.append(fugi.bin_ndarray(np.flip(cPECD_R_all),new_shape=(15,), operation='mean').tolist())
    out.append(fugi.bin_ndarray(np.flip(errPECD_R_all),new_shape=(15,), operation='mean').tolist())
    file1.write("theta \t cPECD_R_all \t err \t") 

    if fileSCL and fileSCR is not None:
        out.append(fugi.bin_ndarray(np.flip(cPECD_all),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(errPECD_all),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(cPECD_S_all),new_shape=(15,), operation='mean').tolist())
        out.append(fugi.bin_ndarray(np.flip(errPECD_S_all),new_shape=(15,), operation='mean').tolist())
        file1.write("\t cPECD_all \t err \t cPECD_S_all \t err \t")

file1.write("\n")
np.savetxt(file1, np.array(out).T, fmt="%.4e" ,delimiter='\t')

file1.close() 